In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

#Need py38 environment for this to run

import sys
print("Python version")
print (sys.version)

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot
print(kale.__version__)

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# ---- Create initial population

pop = holo.population.Pop_Illustris()

# ---- Apply population modifiers
redz = cosmo.a_to_z(pop.scafa)

hards = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
]

f_edd = 0.1
all_evols = {}
eccen = 0.8

acc_instance = holo.accretion.Accretion(accmod='Siwek22', f_edd = f_edd, eccen=eccen, subpc=True)
evo = holo.evolution.Evolution(pop, hards, nsteps = 100, debug=True, acc=acc_instance)
evo.evolve()


In [ ]:
#Take a few random numbers and plot their mass evolutions as a function of time
import random
fs = 20

n_mbhb = evo.shape[0]

n_mbhb_to_plot = 2
inds_random_mbhb = []

p = 0
while p < n_mbhb_to_plot:
    x = random.randint(0,n_mbhb)
#     inds_random_mbhb.append(x)
#     p+=1
    inds_t_pos = evo.tlook[x] > 0
    if len(evo.tlook[x][inds_t_pos]) > 5:
        inds_random_mbhb.append(x)
        p += 1


fig, axs = plt.subplots(int(n_mbhb_to_plot),2,figsize=(15,15))
fig.suptitle('Separations and Masses of MBHBs', fontsize=fs)
k = 0
for i in range(0,int(n_mbhb_to_plot)):
    #for j in range(0,int(n_mbhb_to_plot/2)):
    inds_t_pos = evo.tlook[inds_random_mbhb[k]] > 0
    time = (evo.tlook[inds_random_mbhb[k]][inds_t_pos])
    masses = (evo.mass[inds_random_mbhb[k]][inds_t_pos])
    separation = (evo.sepa[inds_random_mbhb[k]][inds_t_pos])
    dadt = -(evo.dadt[inds_random_mbhb[k]][inds_t_pos])
    #PLOT SEPARATIONS
    j = 0
    axs[i][j].loglog(separation/PC, (separation/dadt)/GYR, '*-')
    axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
    #axs[i][j].set_ylabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
    axs[i][j].set_ylabel(r'$t_{\rm{coal}} \, [\rm{GYR}]$', fontsize=fs)
    #axs[i][j].invert_xaxis()
    #axs[i][j].legend(fontsize=fs)
    plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
    #PLOT MASSES
    j = 1
    if masses.T[0][0] >= masses.T[1][0]:
        axs[i][j].loglog((separation/PC), masses.T[0]/MSOL, '*-', label = 'Primary')
        axs[i][j].loglog(separation/PC, masses.T[1]/MSOL, '*-', label = 'Secondary')
    else:
        axs[i][j].loglog(separation/PC, masses.T[1]/MSOL, '*-', label = 'Primary')
        axs[i][j].loglog(separation/PC, masses.T[0]/MSOL, '*-', label = 'Secondary')
    #axs[i][j].set_xlabel("lookback time [GYR]", fontsize=fs) 
    axs[i][j].set_xlabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
    axs[i][j].set_ylabel(r'$M_{\rm{MBHB}}/M_{\odot}$', fontsize=fs)
    axs[i][j].invert_xaxis()
    axs[i][j].legend(fontsize=fs)
    plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
    k+=1

        
plt.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
